### Revision_LSA vs RLN TEST (Bootstrapping)

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.utils import resample
from sklearn.metrics import (
    confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, 
    roc_auc_score, ConfusionMatrixDisplay, roc_curve, auc, precision_recall_curve,
    balanced_accuracy_score, matthews_corrcoef
)

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import shutil
import os
import random
import time
import csv
import glob
from tqdm import tqdm
from PIL import Image

In [3]:
# GPU setting
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print('GPU available.', device, 'will be used')
else:
    device = torch.device("cpu")
    print('GPU unavailable.', device, 'will be used')

GPU available. cuda:0 will be used


In [12]:
## Bootstrapping by patient ID
def calculate_bootstrap_ci(y_true, y_pred, y_probs, patient_ids, n_iterations=1000, confidence_level=0.95):
    """
    Receives a list of patient IDs, performs patient-level bootstrapping,
    and returns the 95% CI of key metrics as a formatted string.
    """
    y_true = np.array(y_true) # NumPy array: supports 'vectorized operations' that apply conditions or perform operations on all data without loops
    y_pred = np.array(y_pred)
    y_probs = np.array(y_probs)
    patient_ids = np.array(patient_ids)

    unique_patients = np.unique(patient_ids)
    n_patients = len(unique_patients)

    # Dictionary to save metrics
    scores = {
        "Accuracy": [], "Balanced Accuracy": [], "MCC": [],
        "Sensitivity": [], "Specificity": [],
        "PPV": [], "NPV": [],
        "F1 Score (Binary)": [],
        "Precision (Binary)": [],
        "AUC-ROC": [],
        "AUC-PR": [],
        "Precision (Weighted)": [],
        "Recall (Weighted)": [],
        "F1 Score (Weighted)": []
    }

    print(f"[INFO] Starting Patient-level Bootstrapping ({n_iterations} iterations)...")

    ## Bootstrapping Loop
    for _ in tqdm(range(n_iterations), desc="Bootstrapping"):
        # 1. Replacement by Patients unit
        sampled_patients = resample(unique_patients, replace=True, n_samples=n_patients)

        # 2. Index collection of sampled patients
        indices_boot = []
        for pid in sampled_patients:
            indices_boot.extend(np.where(patient_ids == pid)[0])

        y_true_boot = y_true[indices_boot]
        y_pred_boot = y_pred[indices_boot]
        y_probs_boot = y_probs[indices_boot]

        # Exception: Skip in cases where only one class is selected
        if len(np.unique(y_true_boot)) < 2: 
            continue

        # 3. Calculate metrics
        scores["Accuracy"].append(accuracy_score(y_true_boot, y_pred_boot))
        scores["Balanced Accuracy"].append(balanced_accuracy_score(y_true_boot, y_pred_boot))
        scores["MCC"].append(matthews_corrcoef(y_true_boot, y_pred_boot))
        
        # Extract TN, FP, FN, TP
        tn, fp, fn, tp = confusion_matrix(y_true_boot, y_pred_boot, labels=[0, 1]).ravel()
        
        # Sensitivity = Recall = TP / (TP + FN)
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        scores["Sensitivity"].append(sensitivity)

        # Specificity = TN / (TN + FP)
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        scores["Specificity"].append(spec)

        # PPV = Precision = TP / (TP + FP)
        ppv = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        scores["PPV"].append(ppv)
        scores["Precision (Binary)"].append(ppv)
        
        # NPV = TN / (TN + FN)
        npv = tn / (tn + fn) if (tn + fn) > 0 else 0.0
        scores["NPV"].append(npv)
        
        # F1 score (beta =1)
        scores["F1 Score (Binary)"].append(f1_score(y_true_boot, y_pred_boot, average='binary', zero_division=1))

        scores["Precision (Weighted)"].append(precision_score(y_true_boot, y_pred_boot, average='weighted', zero_division=1))
        scores["Recall (Weighted)"].append(recall_score(y_true_boot, y_pred_boot, average='weighted', zero_division=1))
        scores["F1 Score (Weighted)"].append(f1_score(y_true_boot, y_pred_boot, average='weighted', zero_division=1))

        try:
            scores["AUC-ROC"].append(roc_auc_score(y_true_boot, y_probs_boot))

            precision_curve, recall_curve, _ = precision_recall_curve(y_true_boot, y_probs_boot)
            scores["AUC-PR"].append(auc(recall_curve, precision_curve))

        except ValueError:
            continue

    # 4. CI calculation and formatting
    alpha = (1.0 - confidence_level) / 2.0
    ci_results = {}
    for metric_name, metric_scores in scores.items():
        if len(metric_scores) == 0:
            ci_results[metric_name] = "N/A"
            continue
        
        lower_bound = np.percentile(metric_scores, alpha * 100)
        upper_bound = np.percentile(metric_scores, (1.0 - alpha) * 100)
        mean_score = np.mean(metric_scores)
        # results format: mean score ( CI: lower bound ~ upper bound)
        ci_results[metric_name] = f"{mean_score:.6f} ({lower_bound:.6f}-{upper_bound:.6f})"

    print("Bootstrapping completed!")
    return ci_results

In [6]:
transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir2 = '/blue/jkim19/hyunji.jo/AI_cytology/2025/dataset/patients/classification/lsa_rln'

test_datasets = datasets.ImageFolder(os.path.join(data_dir2, 'test'), transform=transforms_test)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=8, shuffle=False) ## shuffle=False

print('Test dataset size:', len(test_datasets))
class_names = test_datasets.classes
print('Classes:', class_names) 

Test dataset size: 490
Classes: ['ln_reactive', 'lsa']


In [7]:
print("Automatically extracting patient IDs form filenames for bootstrapping")

# Ex: '011121_383704_01.tif' -> extract '383704'
pid_pattern = re.compile(r"_(\d{6})")

def extract_pid_from_path(path):
    filename = os.path.basename(path)
    match = pid_pattern.search(filename)
    if match:
        return match.group(1)
    else:
        print(f"PID extraction failed for: {filename}. Using full filename.")
        return filename

patient_ids = [extract_pid_from_path(s[0]) for s in test_datasets.samples]

model_dir = "/blue/jkim19/hyunji.jo/AI_cytology/2025/By_Patients/1120_Revision"
check_csv_path = os.path.join(model_dir, "1208_patient_id_check.csv")
print(f"Saving extracted patient IDs to {check_csv_path} for verification")

try:
    with open(check_csv_path, mode="w", newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["No.", "Original Filename", "Extracted Patient ID"])

        for i, (sample, pid) in enumerate (zip(test_datasets.samples, patient_ids)):
            original_filename = os.path.basename(sample[0])
            writer.writerow([i + 1, original_filename, pid])
    print("Save completed. Please check the file to verify the IDs")
except Exception as e:
    print(f"Error: Failed to save file: {e}") # e: reason for error

total_test_images = len(test_datasets)
print(f"Patient ID list generated. Length: {len(patient_ids)}")

assert len(patient_ids) == total_test_images, \
    f"Error! NOT same length -> Patient ID list ({len(patient_ids)}) != dataset size ({total_test_images})"
print("Verification successful. Ready for testing!")

Automatically extracting patient IDs form filenames for bootstrapping
Saving extracted patient IDs to /blue/jkim19/hyunji.jo/AI_cytology/2025/By_Patients/1120_Revision/1208_patient_id_check.csv for verification
Save completed. Please check the file to verify the IDs
Patient ID list generated. Length: 490
Verification successful. Ready for testing!


In [8]:
model_dir = "/blue/jkim19/hyunji.jo/AI_cytology/2025/By_Patients/0611_seed123/model_save"
model_files = sorted(glob.glob(os.path.join(model_dir, "model_fold_*_best.pth")))

if not model_files:
    print("Error! files were not founded")
    exit()
print(f"Found {len(model_files)} models for testing")

fold_dir = "/blue/jkim19/hyunji.jo/AI_cytology/2025/By_Patients/1120_Revision"
result_dir = os.path.join(fold_dir, "result_plot_REVISED_final")
os.makedirs(result_dir, exist_ok=True)

Found 10 models for testing


In [13]:
results = []

for model_path in model_files:
    fold_name = os.path.basename(model_path).replace("_best.pth", "")
    print(f"\n[INFO] --- Testing Model: {fold_name} ---")

    model = models.resnet50(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(model.fc.in_features, 1)
    model = model.to(device)
    
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()

    preds_array = []
    y_test = []
    y_probs = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_dataloader, desc=f"Inference {fold_name}"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze(1)
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).float()

            preds_array.extend(preds.cpu().numpy())
            y_test.extend(labels.cpu().numpy())
            y_probs.extend(probs.cpu().numpy())

    # Perform Bootstrapping by patient id & Save
    ci_results_str = calculate_bootstrap_ci(y_test, preds_array, y_probs, patient_ids, n_iterations=1000)

    accuracy_str = ci_results_str["Accuracy"]
    balanced_acc_str = ci_results_str["Balanced Accuracy"]
    mcc_str = ci_results_str["MCC"]
    sensitivity_str = ci_results_str["Sensitivity"]
    specificity_str = ci_results_str["Specificity"]
    ppv_str = ci_results_str["PPV"]
    npv_str = ci_results_str["NPV"]
    f1_binary_str = ci_results_str["F1 Score (Binary)"]

    precision_binary_str = ci_results_str["Precision (Binary)"]
    auc_roc_str = ci_results_str["AUC-ROC"]
    auc_pr_str = ci_results_str["AUC-PR"]

    precision_weighted_str = ci_results_str["Precision (Weighted)"]
    recall_weighted_str = ci_results_str["Recall (Weighted)"]
    f1_weighted_str = ci_results_str["F1 Score (Weighted)"]

    print(f"[RESULT WITH 95% CI] {fold_name}")
    print(f"  - Bal Acc: {balanced_acc_str}, MCC: {mcc_str}")
    print(f"  - F1(bin): {f1_binary_str}, F1(weighted): {f1_weighted_str}")
    print(f"  - AUC-ROC: {auc_roc_str}, AUC-PR: {auc_pr_str}")
    
    results.append([
        fold_name, 
        accuracy_str, 
        balanced_acc_str, 
        mcc_str, 
        sensitivity_str,
        specificity_str, 
        ppv_str,
        npv_str,
        precision_binary_str,
        auc_roc_str,
        auc_pr_str, 
        f1_binary_str,
        precision_weighted_str, 
        recall_weighted_str, 
        f1_weighted_str
    ])

    incorrect_filenames = []
    for i, (y_pred, y) in enumerate(zip(preds_array, y_test)):
        if y_pred != y:
            filename = test_datasets.samples[i][0]
            incorrect_filenames.append((filename, int(y_pred), int(y)))
    incorrect_csv_path = os.path.join(result_dir, f"{fold_name}_incorrect_predictions.csv")
    with open(incorrect_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["File Name", "Prediction", "Actual"])
        for f, pred, actual in incorrect_filenames:
            writer.writerow([os.path.basename(f), class_names[pred], class_names[actual]])

    cm_raw = confusion_matrix(y_test, preds_array)
    cm_norm = confusion_matrix(y_test, preds_array, normalize='true')

    # Standard CM
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_raw, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(6,6))
    disp.plot(ax=ax, cmap='magma', values_format='d')
    plt.title(f"Confusion Matrix - {fold_name}")
    plt.savefig(os.path.join(result_dir, f"{fold_name}_confusion_matrix.png"), bbox_inches='tight')
    plt.close()

    # Normalized CM
    disp_norm = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(6,6))
    disp_norm.plot(ax=ax, cmap='magma', values_format='.6f')
    plt.title(f"Normalized Confusion Matrix - {fold_name}")
    plt.savefig(os.path.join(result_dir, f"{fold_name}_confusion_matrix_normalized.png"), bbox_inches='tight')
    plt.close()
    print(f"[INFO] Plots saved for {fold_name}")

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_probs)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(5,5))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.6f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {fold_name}')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig(os.path.join(result_dir, f"{fold_name}_roc_curve.png"), bbox_inches='tight')
    plt.close()

    # PR Curve
    precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_probs)
    pr_auc = auc(recall_curve, precision_curve)
    plt.figure(figsize=(6,6))
    plt.plot(recall_curve, precision_curve, color="blue", lw=2, label=f'PR Curve (AUC = {pr_auc:.6f})')
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f'Precision-Recall Curve - {fold_name}')
    plt.legend(loc="upper right")
    plt.grid(True)
    plt.savefig(os.path.join(result_dir, f"{fold_name}_pr_curve.png"), bbox_inches='tight')
    plt.close()

# ==================================================================
# Final results with 95% CI
# ==================================================================
result_csv_path = os.path.join(model_dir, "1208_ResNet50_10Fold_Test_Results_all_CI_Final.csv")
with open(result_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    header = [
        "Model (Fold)", 
        "Accuracy (95% CI)", 
        "Balanced Accuracy (95% CI)", 
        "MCC (95% CI)", 
        "Sensitivity (95% CI)",
        "Specificity (95% CI)", 
        "PPV (95% CI)",
        "NPV (95% CI)",
        "Precision (95% CI)",
        "AUC-ROC (95% CI)",
        "AUC-PR (95% CI)", 
        "F1 Score (Binary) (95% CI)", 
        "Precision (Weighted) (95% CI)", 
        "Recall (Weighted) (95% CI)", 
        "F1 Score (Weighted) (95% CI)"
    ]
    writer.writerow(header)
    writer.writerows(results)

print(f"\n[SUCCESS] All 10-fold test results with 95% CI (including Weighted metrics) saved to {result_csv_path}")





[INFO] --- Testing Model: model_fold_10 ---


Inference model_fold_10: 100%|██████████| 62/62 [00:54<00:00,  1.13it/s]


[INFO] Starting Patient-level Bootstrapping (1000 iterations)...


Bootstrapping: 100%|██████████| 1000/1000 [00:05<00:00, 167.68it/s]


Bootstrapping completed!
[RESULT WITH 95% CI] model_fold_10
  - Bal Acc: 0.944092 (0.901353-0.981037), MCC: 0.887963 (0.806725-0.959292)
  - F1(bin): 0.950176 (0.899592-0.983310), F1(weighted): 0.945983 (0.906164-0.980861)
  - AUC-ROC: 0.988396 (0.967480-0.999846), AUC-PR: 0.991519 (0.971422-0.999876)
[INFO] Plots saved for model_fold_10

[INFO] --- Testing Model: model_fold_1 ---


...

Bootstrapping completed!
[RESULT WITH 95% CI] model_fold_8
  - Bal Acc: 0.898187 (0.827370-0.961463), MCC: 0.810399 (0.686868-0.922072)
  - F1(bin): 0.916030 (0.834601-0.972493), F1(weighted): 0.904902 (0.833523-0.964281)
  - AUC-ROC: 0.974715 (0.941557-0.996100), AUC-PR: 0.978685 (0.938610-0.997574)
[INFO] Plots saved for model_fold_8

[INFO] --- Testing Model: model_fold_9 ---


Inference model_fold_9: 100%|██████████| 62/62 [00:43<00:00,  1.44it/s]


[INFO] Starting Patient-level Bootstrapping (1000 iterations)...


Bootstrapping: 100%|██████████| 1000/1000 [00:05<00:00, 170.88it/s]


Bootstrapping completed!
[RESULT WITH 95% CI] model_fold_9
  - Bal Acc: 0.879646 (0.793538-0.955974), MCC: 0.769976 (0.615595-0.909892)
  - F1(bin): 0.899763 (0.806875-0.966734), F1(weighted): 0.886052 (0.805442-0.960384)
  - AUC-ROC: 0.959904 (0.904040-0.992173), AUC-PR: 0.967666 (0.911858-0.995461)
[INFO] Plots saved for model_fold_9

[SUCCESS] All 10-fold test results with 95% CI (including Weighted metrics) saved to /blue/jkim19/hyunji.jo/AI_cytology/2025/By_Patients/0611_seed123/model_save/1208_ResNet50_10Fold_Test_Results_all_CI_Final.csv
